### Ensure no other containers are running for dev container, if they are stop and remove them


In [1]:
%%writefile .devcontainer/.dockerignore
**/.git
**/.vscode
**/.idea
**/__pycache__
**/*.pyc
**/*.pyo
**/*.pyd
**/*.swp
**/venv
**/env

Overwriting .devcontainer/.dockerignore


In [2]:
%%writefile .env
ENV_NAME=docker_dev_template
COMPOSE_PROJECT_NAME=docker_dev_template
PYTHON_VER=3.10
QUARTO_VER=1.3.450
JUPYTER_PORT=8889
TENSORBOARD_PORT=6006

COMPOSE_PROJECT_NAME=docker_dev_template

CONDA_ENV_NAME=docker_dev_template
ENV_NAME=docker_dev_template
PYTHON_VER=3.10
QUARTO_VER=1.3.450
JUPYTER_LOG_LEVEL=DEBUG
JUPYTER_PLATFORM_DIRS=1

XLA_PYTHON_CLIENT_PREALLOCATE=true
XLA_PYTHON_CLIENT_ALLOCATOR=platform
XLA_PYTHON_CLIENT_MEM_FRACTION=0.95
JAX_PLATFORM_NAME=gpu
XLA_FLAGS=--xla_force_host_platform_device_count=1 --xla_gpu_deterministic_reductions
JAX_DISABLE_JIT=false
JAX_ENABLE_X64=false
TF_FORCE_GPU_ALLOW_GROWTH=false
JAX_PREALLOCATION_SIZE_LIMIT_BYTES=8589934592

HOST_JUPYTER_PORT=8889
HOST_TENSORBOARD_PORT=6006
HOST_EXPLAINER_PORT=8050
HOST_STREAMLIT_PORT=8501

Overwriting .env


In [3]:
%%writefile .devcontainer/.env
COMPOSE_PROJECT_NAME=docker_dev_template

CONDA_ENV_NAME=docker_dev_template
ENV_NAME=docker_dev_template
PYTHON_VER=3.10
QUARTO_VER=1.3.450
JUPYTER_LOG_LEVEL=DEBUG
JUPYTER_PLATFORM_DIRS=1

XLA_PYTHON_CLIENT_PREALLOCATE=true
XLA_PYTHON_CLIENT_ALLOCATOR=platform
XLA_PYTHON_CLIENT_MEM_FRACTION=0.95
JAX_PLATFORM_NAME=gpu
XLA_FLAGS=--xla_force_host_platform_device_count=1 --xla_gpu_deterministic_reductions
JAX_DISABLE_JIT=false
JAX_ENABLE_X64=false
TF_FORCE_GPU_ALLOW_GROWTH=false
JAX_PREALLOCATION_SIZE_LIMIT_BYTES=8589934592

HOST_JUPYTER_PORT=8889
HOST_TENSORBOARD_PORT=6006
HOST_EXPLAINER_PORT=8050
HOST_STREAMLIT_PORT=8501

Overwriting .devcontainer/.env


In [4]:
%%writefile .devcontainer/devcontainer.json
{
    "name": "docker_dev_template",
    "build": {
        "dockerfile": "Dockerfile",
        "context": "..",
        "args": {
            "ENV_NAME": "docker_dev_template",
            "PYTHON_VER": "${localEnv:PYTHON_VER:3.10}",
            "QUARTO_VER": "${localEnv:QUARTO_VER:1.3.450}"
        }
    },
    "customizations": {
        "vscode": {
            "settings": {
                "python.defaultInterpreterPath": "/opt/conda/envs/docker_dev_template/bin/python",
                "python.pythonPath": "/opt/conda/envs/docker_dev_template/bin/python",
                "terminal.integrated.profiles.linux": {
                    "bash": {
                        "path": "bash",
                        "args": ["-l"]
                    }
                },
                "terminal.integrated.defaultProfile.linux": "bash",
                "terminal.integrated.cwd": "/workspace"
            },
            "extensions": [
                "ms-python.python",
                "ms-python.vscode-pylance",
                "ms-toolsai.jupyter",
                "ms-toolsai.jupyter-renderers",
                "quarto.quarto",
                "purocean.drawio-preview",
                "redhat.vscode-yaml",
                "yzhang.markdown-all-in-one",
                "ms-azuretools.vscode-docker",
                "ms-vscode-remote.remote-containers",
                "github.vscode-github-actions",
                "GitHub.copilot",
                "GitHub.vscode-pull-request-github",
                "njpwerner.autodocstring",
                "aaron-bond.better-comments",
                "oderwat.indent-rainbow",
                "mtxr.sqltools",
                "ms-mssql.mssql",
                "Leonardo16.nvidia-gpu",
                "mechatroner.rainbow-csv"
            ]
        }
    },
    "workspaceFolder": "/workspace",
    "mounts": [
        "source=${localWorkspaceFolder},target=/workspace,type=bind,consistency=cached"
    ],
    "remoteEnv": {
        "MY_VAR": "${localEnv:MY_VAR:test_var}"
    },
    "runArgs": [
        "--gpus", "all",
        "--env-file", ".devcontainer/.env"
    ],
    "postCreateCommand": "conda run -n docker_dev_template "
}

Overwriting .devcontainer/devcontainer.json


In [5]:
%%writefile .devcontainer/Dockerfile
FROM nvidia/cuda:12.3.2-cudnn9-devel-ubuntu22.04

ARG ENV_NAME=docker_dev_template
ARG PYTHON_VER=3.10
ARG JAX_PREALLOCATE=true
ARG JAX_MEM_FRAC=0.95
ARG JAX_ALLOCATOR=platform
ARG JAX_PREALLOC_LIMIT=8589934592

ENV DEBIAN_FRONTEND=noninteractive
ENV CONDA_DIR=/opt/conda
ENV PATH=$CONDA_DIR/bin:$PATH
ENV LD_LIBRARY_PATH=/opt/conda/envs/${ENV_NAME}/lib:$LD_LIBRARY_PATH

# 1. Install system deps
RUN apt-get update --fix-missing && \
 apt-get install -y --no-install-recommends \
   software-properties-common \
   wget \
   bzip2 \
   ca-certificates \
   apt-transport-https \
   gnupg2 \
   curl \
   git \
   vim \
   python3-pip \
   python3-dev \
   pkg-config \
   build-essential \
   autoconf \
   automake \
   libtool \
   m4 \
   cmake && \
 rm -rf /var/lib/apt/lists/*

# 4. Miniconda installation
RUN wget --quiet https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda.sh && \
 /bin/bash ~/miniconda.sh -b -p $CONDA_DIR && \
 rm ~/miniconda.sh && \
 conda clean -a -y

# 5. Create the conda environment
COPY .devcontainer/environment.yml /tmp/environment.yml
RUN sed -i "s/^name: .*/name: $ENV_NAME/" /tmp/environment.yml && \
 sed -i "s/^- python=.*/- python=$PYTHON_VER/" /tmp/environment.yml && \
 conda env create -f /tmp/environment.yml && \
 conda clean -a -y

# 6. Install PyJAGS with cstdint header fix
RUN . $CONDA_DIR/etc/profile.d/conda.sh && \
 conda activate ${ENV_NAME} && \
 echo '#!/usr/bin/env python3' > /tmp/install_pyjags.py && \
 echo 'import os' >> /tmp/install_pyjags.py && \
 echo 'os.environ["CPPFLAGS"] = "-include cstdint"' >> /tmp/install_pyjags.py && \
 echo 'print("Installing PyJAGS with cstdint header fix...")' >> /tmp/install_pyjags.py && \
 echo 'os.system("CC=x86_64-conda-linux-gnu-cc CXX=x86_64-conda-linux-gnu-c++ pip install --no-build-isolation pyjags==1.3.8")' >> /tmp/install_pyjags.py && \
 echo 'print("PyJAGS installation complete!")' >> /tmp/install_pyjags.py && \
 python /tmp/install_pyjags.py && \
 python -c "import sys; print('Attempting to import PyJAGS...')" && \
 python -c "import pyjags; print('PyJAGS OK →', pyjags.__version__)" || \
 (conda install -c conda-forge -y pyjags=1.3 && \
 python -c "import pyjags; print('PyJAGS OK (conda) →', pyjags.__version__)") && \
 rm -f /tmp/install_pyjags.py

# 7. Ensure the environment auto-activates
RUN echo ". $CONDA_DIR/etc/profile.d/conda.sh && conda activate $ENV_NAME" \
   >> /etc/bash.bashrc

# 8. Tune JAX GPU behavior
ENV XLA_PYTHON_CLIENT_PREALLOCATE=${JAX_PREALLOCATE}
ENV XLA_PYTHON_CLIENT_MEM_FRACTION=${JAX_MEM_FRAC}
ENV XLA_PYTHON_CLIENT_ALLOCATOR=${JAX_ALLOCATOR}
ENV JAX_PLATFORM_NAME=gpu
ENV XLA_FLAGS="--xla_force_host_platform_device_count=1"
ENV JAX_DISABLE_JIT=false
ENV JAX_ENABLE_X64=false
ENV TF_FORCE_GPU_ALLOW_GROWTH=false
ENV JAX_PREALLOCATION_SIZE_LIMIT_BYTES=${JAX_PREALLOC_LIMIT}

# 9. Install JAX CUDA-enabled wheel
RUN . $CONDA_DIR/etc/profile.d/conda.sh && \
 conda activate $ENV_NAME && \
 pip install --no-cache-dir --upgrade "jax[cuda12]==0.5.2" \
   -f https://storage.googleapis.com/jax-releases/jax_releases.html

# 10. Install IPython kernel for Jupyter
RUN . $CONDA_DIR/etc/profile.d/conda.sh && \
 conda activate ${ENV_NAME} && \
 python -m ipykernel install --name ${ENV_NAME} \
   --display-name "Python (${ENV_NAME})"

WORKDIR /workspace

# 11. Deduplicate LD_LIBRARY_PATH on shell startup
RUN echo 'export LD_LIBRARY_PATH=$(printf "%s" "$LD_LIBRARY_PATH" \
 | tr ":" "\n" | awk "!seen[$0]++" | paste -sd ":")' \
 >> /etc/profile.d/clean_ld.sh

Overwriting .devcontainer/Dockerfile


In [6]:
%%writefile .devcontainer/environment.yml
# .devcontainer/environment.yml
name: docker_dev_template
channels:
  - conda-forge
  - defaults

dependencies:
  # Core language & kernels
  - python=3.10
  - jupyterlab>=3.0.0
  - ipykernel
  - ipywidgets>=8.0.0
  - optuna=4.3.0

  # Data & ML libraries (conda)
  - numpy>=1.20.0
  - pandas>=1.2.0
  - scipy>=1.7.0
  - scikit-learn>=1.4.2
  - matplotlib>=3.4.0
  - seaborn>=0.11.0
  - xgboost>=1.5.0
  - lightgbm>=3.3.0
  - catboost>=1.0.0

  # Node for widgets & front-end tools
  - nodejs>=18.0.0

  # Bayesian & JAX-adjacent (conda)
  - numpyro>=0.18.0
  - pymc>=5.0.0
  - arviz>=0.14.0
  - statsmodels>=0.13.0

  # JAGS libraries as conda fallback
  - jags>=4.3.0
  - pybind11>=2.9.0  # Compatible with Python 3.10
  - cxx-compiler
  - c-compiler
  - make
  - pkg-config
  
  # Streamlit for interactive dashboards
  - streamlit>=1.20.0

  # Utilities & extras (conda)
  - nutpie>=0.14.3 
  - tabulate>=0.9.0
  - tqdm>=4.67.0
  - pyarrow>=12.0.0
  - sqlalchemy
  - mysql-connector-python

  # Development tooling
  - pytest>=7.0.0
  - black>=23.0.0
  - isort>=5.0.0
  - flake8>=5.0.0
  - mypy>=1.0.0


  # Bayesian & JAX-adjacent (conda)
  - cmdstanpy>=1.2           # Stan front-end :contentReference[oaicite:5]{index=5}
  - cmdstan>=2.36            # CmdStan C++ backend :contentReference[oaicite:6]{index=6}
  - tensorflow-probability   # Probabilistic methods on TF :contentReference[oaicite:8]{index=8}


  # Ensure pip is available
  - pip

  # Pure-pip installs (no JAX here! it's handled in Dockerfile)
  - pip:
    - pytensor>=2.18.3
    - shap>=0.40.0
    - shapiq>=0.1.0
    - explainerdashboard>=0.3.0
    - shapash[report]>=2.3.0
    - bayesian-optimization
    - pretty_errors
    - gdown
    - invoke>=2.2
    # - tensorflow>=2.16         # Required for TFP :contentReference[oaicite:9]{index=9}
    # - tf-keras>=2.18.0 
    - tf-nightly
    - tf-keras-nightly

Overwriting .devcontainer/environment.yml


In [7]:
%%writefile .devcontainer/gpu_verify.py
#!/usr/bin/env python3
"""
Verify that the GPU is accessible and JAX is correctly configured.
This script is used during container startup.
"""

import sys

def check_gpu():
    print("Checking GPU availability...")
    try:
        import jax
        jax.config.update('jax_platform_name', 'gpu')
        
        # Get device count and details
        devices = jax.devices()
        device_count = len(devices)
        print(f"JAX version: {jax.__version__}")
        print(f"Available devices: {device_count}")
        
        for i, device in enumerate(devices):
            print(f"Device {i}: {device}")
        
        if device_count == 0 or 'gpu' not in str(devices[0]).lower():
            print("WARNING: No GPU devices found by JAX!")
            return False
        
        # Check CUDA configuration
        import jax.tools.jax_jit
        jit_info = jax.tools.jax_jit.get_jax_jit_flags()
        print(f"JIT configuration: {jit_info}")
        
        # Run a simple GPU computation
        print("Running a test computation on GPU...")
        import numpy as np
        x = np.ones((1000, 1000))
        result = jax.numpy.sum(x, axis=0)
        print(f"Test computation result shape: {result.shape}")
        
        print("JAX GPU verification completed successfully!")
        return True
    
    except ImportError:
        print("JAX not found! Make sure JAX is installed with GPU support.")
        return False
    except Exception as e:
        print(f"Error during GPU verification: {e}")
        return False

if __name__ == "__main__":
    success = check_gpu()
    if not success:
        print("WARNING: GPU verification failed!")
        # Not exiting with error to allow container to start anyway
        # sys.exit(1)
    else:
        sys.exit(0) 

Overwriting .devcontainer/gpu_verify.py


In [8]:
%%writefile .devcontainer/jags_verify.py
#!/usr/bin/env python3
"""
Verify that PyJAGS is correctly installed and working.
This script is used by the Docker container health check.
"""

import sys
try:
    import pyjags
    print(f"PyJAGS version: {pyjags.__version__}")
    
    # Create a simple model to verify that PyJAGS works
    code = """
    model {
        # Likelihood
        y ~ dnorm(mu, 1/sigma^2)
        
        # Priors
        mu ~ dnorm(0, 0.001)
        sigma ~ dunif(0, 100)
    }
    """
    
    # Sample data
    data = {'y': 0.5}
    
    # Initialize model with data
    model = pyjags.Model(code, data=data, chains=1, adapt=100)
    print("JAGS model initialized successfully!")
    
    # Sample from the model
    samples = model.sample(200, vars=['mu', 'sigma'])
    print("JAGS sampling completed successfully!")
    
    # Verify the samples
    mu_samples = samples['mu']
    sigma_samples = samples['sigma']
    print(f"mu mean: {mu_samples.mean():.4f}")
    print(f"sigma mean: {sigma_samples.mean():.4f}")
    
    print("PyJAGS verification completed successfully!")
    sys.exit(0)
    
except ImportError:
    print("PyJAGS not found!")
    sys.exit(1)
except Exception as e:
    print(f"Error during PyJAGS verification: {e}")
    sys.exit(1) 

Overwriting .devcontainer/jags_verify.py


In [9]:
%%writefile .devcontainer/pyjags_patch.py
#!/usr/bin/env python3
import os
import sys

def patch_pyjags_sources():
    print("Downloading and patching PyJAGS source...")
    os.system("pip download --no-binary :all: pyjags==1.3.8")
    os.system("tar -xzf pyjags-1.3.8.tar.gz")
    os.chdir("pyjags-1.3.8")
    
    # Add cstdint include to all cpp files
    for root, dirs, files in os.walk("src"):
        for file in files:
            if file.endswith(".cpp") or file.endswith(".h"):
                filepath = os.path.join(root, file)
                with open(filepath, 'r') as f:
                    content = f.read()
                if "#include <cstdint>" not in content:
                    with open(filepath, 'w') as f:
                        f.write("#include <cstdint>\n" + content)
                    print(f"Patched {filepath}")
    
    # Build and install
    os.system("pip install --no-build-isolation .")
    print("PyJAGS installation complete!")
    return 0

if __name__ == "__main__":
    sys.exit(patch_pyjags_sources()) 

Overwriting .devcontainer/pyjags_patch.py


In [10]:
%%writefile docker-compose.yml
# docker-compose.yml
services:
  datascience:
    build:
      context: .
      dockerfile: .devcontainer/Dockerfile
      args:
        ENV_NAME: ${CONDA_ENV_NAME:-docker_dev_template}
        PYTHON_VER: ${PYTHON_VER:-3.10}
        JAX_PREALLOCATE: ${XLA_PYTHON_CLIENT_PREALLOCATE:-true}
        JAX_MEM_FRAC: ${XLA_PYTHON_CLIENT_MEM_FRACTION:-0.95}
        JAX_ALLOCATOR: ${XLA_PYTHON_CLIENT_ALLOCATOR:-platform}
        JAX_PREALLOC_LIMIT: ${JAX_PREALLOCATION_SIZE_LIMIT_BYTES:-8589934592}

    image: "${CONDA_ENV_NAME:-docker_dev_template}-datascience:latest"
    # (Removed explicit container_name to avoid “already in use” conflicts.)

    gpus: all

    env_file:
      - .env

    environment:
      - ENV_NAME=${CONDA_ENV_NAME}
      - PYTHON_VER=${PYTHON_VER}
      - NVIDIA_VISIBLE_DEVICES=all
      - NVIDIA_DRIVER_CAPABILITIES=compute,utility,graphics,display
      - JAX_PLATFORM_NAME=${JAX_PLATFORM_NAME}
      - XLA_PYTHON_CLIENT_PREALLOCATE=${XLA_PYTHON_CLIENT_PREALLOCATE}
      - XLA_PYTHON_CLIENT_ALLOCATOR=${XLA_PYTHON_CLIENT_ALLOCATOR}
      - XLA_PYTHON_CLIENT_MEM_FRACTION=${XLA_PYTHON_CLIENT_MEM_FRACTION}
      - XLA_FLAGS=${XLA_FLAGS}
      - JAX_DISABLE_JIT=${JAX_DISABLE_JIT}
      - JAX_ENABLE_X64=${JAX_ENABLE_X64}
      - TF_FORCE_GPU_ALLOW_GROWTH=${TF_FORCE_GPU_ALLOW_GROWTH}
      - JAX_PREALLOCATION_SIZE_LIMIT_BYTES=${JAX_PREALLOCATION_SIZE_LIMIT_BYTES}

    volumes:
      - .:/workspace

    ports:
      - "${HOST_JUPYTER_PORT:-8889}:8888"
      - "${HOST_TENSORBOARD_PORT:-6006}:6006"
      - "${HOST_EXPLAINER_PORT:-8050}:8050"
      - "${HOST_STREAMLIT_PORT:-8501}:8501"

    command: >
      bash -c "
      conda run -n ${CONDA_ENV_NAME} python -c \"import pyjags; print('PyJAGS version:', pyjags.__version__)\" &&
      conda run -n ${CONDA_ENV_NAME} jupyter lab --ServerApp.crash_info=True --ip 0.0.0.0 --port 8888 --no-browser --allow-root
      "

    healthcheck:
      test: ["CMD", "conda", "run", "-n", "${CONDA_ENV_NAME}", "python", "-c", "import pyjags; print('PyJAGS', pyjags.__version__)"]
      interval: 10s
      timeout: 5s
      retries: 3
      start_period: 30s

Overwriting docker-compose.yml


In [11]:
%%writefile pyproject.toml
[project]
name = "docker_dev_template"
version = "0.1.0"
description = "Hierarchical Bayesian modeling for baseball exit velocity data"
authors = [
  { name = "Marlins Data Science Team" },
]
license = "MIT"
readme = "README.md"

# ─── Restrict to Python 3.10–3.12 ──────────────────────────────
requires-python = ">=3.10,<3.13"

dependencies = [
  "pandas>=1.2.0",
  "numpy>=1.20.0",
  "matplotlib>=3.4.0",
  "scikit-learn>=1.4.2",
  "pymc>=5.0.0",
  "arviz>=0.14.0",
  "statsmodels>=0.13.0",
  "jupyterlab>=3.0.0",
  "seaborn>=0.11.0",
  "tabulate>=0.9.0",
  "shap>=0.40.0",
  "xgboost>=1.5.0",
  "lightgbm>=3.3.0",
  "catboost>=1.0.0",
  "scipy>=1.7.0",
  "shapash[report]>=2.3.0",
  "shapiq>=0.1.0",
  "explainerdashboard>=0.3.0",
  "ipywidgets>=8.0.0",
  "nutpie>=0.7.1",   # new: nutpie backend for PyMC
  "numpyro>=0.18.0,<1.0.0",
  "jax==0.6.0",
  "jaxlib==0.6.0",
  "pytensor>=2.18.3",  # explicit version for CUDA support
  "aesara>=2.9.4",     # alternative backend option
  "tqdm>=4.67.0",
  "pyarrow>=12.0.0",
]

[project.optional-dependencies]
dev = [
  "pytest>=7.0.0",
  "black>=23.0.0",
  "isort>=5.0.0",
  "flake8>=5.0.0",
  "mypy>=1.0.0",
]

cuda = [
  "cupy-cuda12x>=12.0.0",  # For CUDA 12.x
]

[tool.pytensor]
# Default configuration for PyTensor
device = "cuda"          # Use CUDA by default if available
floatX = "float32"       # Use float32 by default for better CUDA performance
allow_gc = true          # Allow garbage collection
optimizer = "fast_run"   # Fast run optimization by default

Overwriting pyproject.toml
